<a href="https://colab.research.google.com/github/GabiCEE2/CEE2/blob/main/a20_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Raspagem de Dados Web

Raspagem de dados (***Web Scraping***) é o processo de automatização para  extrair informações de sites da web. Isso envolve acessar as páginas, identificar elementos específicos no código HTML (como textos, imagens ou links) e converter esses dados em um formato estruturado, como tabelas ou arquivos CSV, para possibilitar sua análise ou utilização em aplicações.

## Principais bibliotecas

Para realizar a raspagem, utilizaremos as seguintes bibliotecas:

- `requests`: Para fazer requisições HTTP e obter o HTML das páginas.

- `BeautifulSoup`: Para analisar e navegar pelo HTML obtido.

Instale essas bibliotecas usando o seguinte comando:

```
pip install requests
pip install beautifulsoup4
```

Uma vez instalado, você pode importar essas bibliotecas com os seguintes comandos:
```python
import requests
from bs4 import BeautifulSoup
```



## Requisição

  Para faze a requisição, vamos utilizar a seguinte função:

  ```python
  requests.get(url)
  ```

  Quando você chama essa função, o Python envia uma solicitação HTTP para o servidor associado à URL fornecida. O servidor responde com um status code (indicando sucesso, erro, etc.) e, normalmente, com o conteúdo solicitado, como HTML ou JSON.

***Argumentos:***

  - `url` (obrigatório): A URL para onde será enviada a solicitação GET.

  - `params` (opcional): Um dicionário de parâmetros de consulta para ser incluído na URL.

  - `headers` (opcional): Cabeçalhos HTTP personalizados, como User-Agent.

  - `timeout` (opcional): Tempo limite para a resposta do servidor, em segundos.

***Retorno:***

A função retorna um objeto da classe `Response`, que contém informações sobre a resposta do servidor, incluindo:

  - `response.status_code`: Código de status HTTP (200 para sucesso, 404 para "não encontrado", etc.).
  - `response.text`: Conteúdo da resposta como texto (usado para HTML).
  - `response.json()`: Converte o conteúdo da resposta para um objeto JSON, se aplicável.
  - `response.content`: Conteúdo bruto da resposta, em bytes.

  Obs: Se ocorrer um erro, ela gera uma exceção automaticamente.

***Exemplo:***

A célula abaixo apresenta um exemplo de como o uso dessa função pode ser implementada.

In [ ]:
import requests

# Função para obter o conteúdo HTML de uma URL
def obter_html(url):
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()  # Lança uma exceção se o código de status HTTP indicar erro (ex: 404, 500).
        return resposta.text  # Retorna apenas o HTML
    except requests.RequestException as e: # Caso ocorra algum erro (como conexão recusada, timeout, erro de HTTP, etc.), imprime uma mensagem e retorna None.
        print(f"Erro ao acessar a URL: {e}")
        return None

Vamos então analisar o comportamento para algumas páginas comuns. Nesses exemplos, vamos apenas apresentar os primeiros 200 caracteres de cada página.

In [ ]:
resposta = obter_html("https://www.google.com")
print(resposta[:200])

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many speci


In [ ]:
resposta = obter_html("https://g1.globo.com/")
print(resposta[:200])

<!DOCTYPE HTML><html lang="pt-br" class> <head><meta charset="utf-8"><meta http-equiv="x-ua-compatible" content="ie=edge,chrome=1"><meta name="viewport" content="width=device-width, initial-scale=1"><


Desta forma, podemos ter acesso ao código html da página, mas isso não parece diretamente muito útil. No entanto, a biblioteca `BeautifulSoup` pode nos ajudar organizando as informações contidas dentro de uma página.

## Estruturação

A estruturação da saída gerada pela função `BeautifulSoup` organiza o conteúdo HTML em uma árvore de elementos DOM (*Document Object Model*). Isso permite navegar pelo conteúdo como se fosse um objeto hierárquico, onde cada nó representa um elemento HTML (tags, atributos, texto, etc.).

Vamos olhar um exemplo fictício de HTML bruto (suponha que tenha sido obtido por request):

```html
<html>
  <head>
    <title>Exemplo de Página</title>
  </head>
  <body>
    <h1>Bem-vindo ao Tutorial</h1>
    <p class="intro">Este é um exemplo simples de raspagem.</p>
    <a href="https://exemplo.com">Leia mais</a>
  </body>
</html>
```

Após rodar a requisição dessa página, vamos receber (via função `.text`) algo do tipo:
```python
"<html><head><title>Exemplo de Página</title></head><body><h1>Bem-vindo ao Tutorial</h1><p class=\"intro\">Este é um exemplo simples de raspagem.</p><a href=\"https://exemplo.com\">Leia mais</a></body></html>"
```

A função `BeautifulSoup` então é chamada passando a página como um argumento:

```python
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content)
```

Desta forma, quando o conteúdo é analisado com `BeautifulSoup`, ele cria uma árvore hierárquica (`soup` no exemplo), onde:

- O elemento `<html>` é a raiz.
- Filhos são os elementos contidos diretamente dentro de outro elemento (como `<head>` e `<body>` no `<html>`).
- Textos, atributos e subelementos são acessíveis como propriedades dos nós.


***Argumentos:***

- `html_content` (obrigatório): O código HTML ou XML bruto que será analisado. Normalmente, é obtido com bibliotecas como requests.

- `parser` (opcional): Especifica o analisador (parser) a ser usado para processar o HTML. Alguns exemplos:
  - 'html.parser': Parser embutido no Python. Funciona bem para a maioria dos casos.
  - 'xml': Para processar documentos XML.

***Retorno:***

  A função retorna um objeto da classe `BeautifulSoup` que representa o HTML como uma árvore de elementos, onde cada nó é um objeto que pode ser manipulado, incluindo:

  - `prettify()`: Apresenta o código em um formato estruturado;

  - `find(tag, attributes)`: Encontra a primeira ocorrência de uma tag.  

  - `find_all(tag, attributes)`: Encontra todas as ocorrências de uma tag.

  - `get_text(strip=True)`: Extrai o texto de dentro de uma tag, removendo espaços extras.


***Navegação:***

1. Navegação Direta: Você pode acessar diretamente elementos específicos, como filhos imediatos:

  ```python
  soup = BeautifulSoup(html_content, 'html.parser')

  # Acessar a tag <html>
  html = soup.html

  # Acessar a tag <body> (filho direto de <html>)
  body = html.body

  # Acessar o título dentro da tag <head>
  titulo = soup.head.title.text  # Saída: "Exemplo de Página"
  ```

2. Localização por Tags:
Use `find()` e `find_all()` para buscar elementos específicos:

  ```python
  # Encontra o primeiro parágrafo <p>
  paragrafo = soup.find('p')
  print(paragrafo.text)  # Saída: "Este é um exemplo simples de raspagem."

  # Encontra todos os links <a>
  links = soup.find_all('a')
  for link in links:
      print(link['href'])  # Saída: "https://exemplo.com"

  ```

3. Navegação por Atributos:
Você pode buscar elementos que possuem atributos específicos, como classes ou IDs:

  ```python
  # Encontrar um elemento com uma classe específica
  paragrafo_intro = soup.find('p', class_='intro')
  print(paragrafo_intro.text)  # Saída: "Este é um exemplo simples de raspagem."
  ```

4. Navegação pelo DOM:
Use propriedades como parent, children, next_sibling, etc., para navegar pela estrutura:

  ```python
  # Navegar para o pai de um elemento
  link = soup.find('a')
  pai_do_link = link.parent
  print(pai_do_link.name)  # Saída: "body"

  # Navegar entre irmãos
  paragrafo = soup.find('p')
  proximo_elemento = paragrafo.next_sibling
  print(proximo_elemento.name)  # Saída: "a" (o próximo irmão é o link <a>)

  ```






## Exemplo: página Clima Tempo

1. Vamos começar abrindo a página de interesse.
  > Exemplo: abra https://www.climatempo.com.br/ e procure por Brasília-DF.

2. Vamos então baixar a página e visualizar em um formato estruturado.
  > Exemplo: foi utilizado https://www.climatempo.com.br/previsao-do-tempo/cidade/61/brasilia-df


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.climatempo.com.br/previsao-do-tempo/cidade/61/brasilia-df"

# Dicionário definindo tipo de exibição como navegador
## você consegue essa informação indo na página e clicando em especionar página
## e depois no campo Network
## Pode ser necessario recarregar a pagina (F5)
## * clique em qualquer item a esquerda, vá em header e busque pelas informacoes.
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8"
}

# Enviar a requisição
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

print( soup.prettify()[:1000] ) # apresenta o código em um formato estruturado

<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <title>
   Previsão do tempo e clima para hoje em Brasília - DF: Clique já!
  </title>
  <meta charset="utf-8"/>
  <!-- This one can help to identify and solve our design issue for Mobile which Google has threatened stopping the ads on Chrome browser. -->
  <meta content="f-v1CUADcZO9RTlI5wOpt11LsuNyyqWC6zHgEG43hQA" name="google-site-verification">
   <!-- and of google site verificatipn -->
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
   <meta content="no-cache, no-store" http-equiv="Cache-Control"/>
   <meta content="no-cache, no-store" http-equiv="Pragma"/>
   <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
   <meta content="Climatempo" name="author"/>
   <meta content="© Climatempo" name="copyright"/>
   <meta content="https://www.climatempo.com.br/previsao-do-tempo/cidade/61/brasilia-df" name="url"/>
   <

In [ ]:
## Código do título da página
print( soup.head.title )      # saída: <title>Previsão do tempo e clima para hoje em Brasília - DF: Clique já!</title>

## Texto do título da página
print( soup.head.title.text ) # saída: Previsão do tempo e clima para hoje em Brasília - DF: Clique já!

<title>Aposentados - Departamento de Estatística</title>
Aposentados - Departamento de Estatística


3. Vamos agora inspecionar a página. Clique com o botão direito e vá em "Inspecionar". No menu que abre, selecione o ícone que corresponde a "Select an element in the page to inspect it - Ctrl+Shit+C".

4. Posicione então o mouse na informação desejada e clique.
  * Note que você será direcionado para o trecho do código HTML que contém a informação que você clicou.
  
  * Desta forma, você pode utilizar as informações de tag, classe e outros para caputar a informação de interesse para o seu código.
  
  Veja os exemplos abaixo.

In [ ]:
## Temperatura mínima
print( soup.find("span", id = "min-temp-1").text )

17°


In [ ]:
## Temperatura máxima
print( soup.find("span", id = "max-temp-1").text )

27°


In [ ]:
##  Previsão de chuva
lista = soup.find_all("span", class_="_margin-l-5")
print(lista) ## note que temos uma lista de retornos

## separando e limpando a informação de interesse
texto_interesse = lista[1].text
import re
texto_interesse = re.sub(r"\s+", " ", texto_interesse) ## "s+, REGEX para sequencia de espaços, tabulações, quebras de linha"

print("\n\nPrevisão de chuva:", texto_interesse )

[<span class="temperature _margin-l-5 -font-13">-</span>, <span class="_margin-l-5">0.0mm -
								0%</span>, <span class="_margin-l-5 _margin-r-5 _inline-block common-sprite sprite-sunshine-sunrise"></span>]


Previsão de chuva: 0.0mm - 0%


## Exemplo: manchetes do G1

- Vamos começar inspecionando a página https://g1.globo.com/.

- Note que as manchetes são apresentadas utilizando a tag `"a"` dentro da classe `"feed-post-link"`.

- Desta forma, podemos utilizar o código abaixo para listar as manchetes:

In [ ]:
import requests
from bs4 import BeautifulSoup

print("Manchetes do G1:")

url = "https://g1.globo.com/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar manchetes
    manchetes = soup.find_all("a", class_="feed-post-link")

    for i, manchete in enumerate(manchetes, start=1):  # Mostrar as 5 primeiras
        titulo = manchete.text
        link = manchete['href']
        print(f"{i}. {titulo} - {link}")

except requests.RequestException as e:
    print(f"Erro na requisição: {e}")

Manchetes do G1:
1. EUA publicam ordem que barra Harvard de admitir alunos estrangeiros - https://g1.globo.com/mundo/noticia/2025/06/04/trump-publica-ordem-que-proibe-harvard-de-admitir-estudantes-estrangeiros.ghtml
2. Moraes manda PF apurar se Zambelli tentou obstruir investigação com fuga - https://g1.globo.com/politica/blog/gerson-camarotti/post/2025/06/04/moraes-determina-abertura-de-inquerito-da-pf-contra-zambelli-para-apurar-obstrucao-de-investigacao-penal.ghtml
3. Deputado italiano pede ao governo do país que Zambelli seja extraditada - https://g1.globo.com/politica/noticia/2025/06/04/deputado-italiano-pede-ao-governo-de-seu-pais-que-zambelli-seja-extraditada.ghtml
4. Putin irá responder a ataque de drones da Ucrânia, diz Trump - https://g1.globo.com/mundo/ucrania-russia/noticia/2025/06/04/putin-ira-responder-a-ataques-de-drones-da-ucrania-diz-trump.ghtml
5. Governo deve mexer no Fundeb e em isenções para rever alta do IOF - https://g1.globo.com/economia/noticia/2025/06/04/gover

## Exemplo: frases do Pensador

Ao inspecionar a página https://www.pensador.com/frases podemos notar que as frases estão distribuídas em tags `"p"` com classes `'frase'`.

O código abaixo lista as 10 primeiras frases encontradas.

In [ ]:
import requests
from bs4 import BeautifulSoup

print("Frases do Pensador:")

url = "https://www.pensador.com/frases/"

try:
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar frases
    frases = soup.find_all("p", class_="frase")
    for i, frase in enumerate(frases[:10], start=1):  # Mostrar as 5 primeiras
        print(f"{i}. {frase.text.strip()}")

except requests.RequestException as e:
    print(f"Erro na requisição: {e}")

Frases do Pensador:
1. Tudo o que um sonho precisa para ser realizado é alguém que acredite que ele possa ser realizado.
2. Imagine uma nova história para sua vida e acredite nela.
3. A amizade desenvolve a felicidade e reduz o sofrimento, duplicando a nossa alegria e dividindo a nossa dor.
4. Nunca deixe ninguém te dizer que não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer e dizem que você também não vai vencer. Se você quer uma coisa, corra atrás.
5. Ser feliz sem motivo é a mais autêntica forma de felicidade.
6. Não existe um caminho para a felicidade. A felicidade é o caminho.
7. Não espere por uma crise para descobrir o que é importante em sua vida.
8. Acredite em si e chegará um dia em que os outros não terão outra escolha senão acreditar com você.
9. Saber encontrar a alegria na alegria dos outros é o segredo da felicidade.
10. A alegria de fazer o bem é a única felicidade verdadeira.


## Exercício 1

Escreva um programa que acessa a página https://www.est.unb.br/ e lista o nome todos os professores aposentados.


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.est.unb.br/index.php?option=com_contact&view=category&id=32&Itemid=172"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

print("Professores aposentados:")

nomes = soup.find_all("h3", class_="item__title")

for nome in nomes:
    print(nome.text.strip())



Professores aposentados:
Aloisio Mayworm
Celso Chiarini
Cibele Queiroz da Silva
Claudete Ruas
Édina Shisue Miazaki
Eduardo Freitas da Silva
Elisete Naomi Abe da Rocha Miranda
Geraldo da Silva e Souza (in memoriam)
Hilton Vieira Machado (in memoriam)
Ismael Pereira
Isolda Hora Accioly (in memoriam)
Jorge de Souza (in memoriam)
Lúcio José Vivaldi
Luis Hernán Rodrigues de Castro
Maria Amélia Biagio
Maurício Pinho Gama
Osiris Turnes
Reynaldo Arcirio (in memoriam)
